In [0]:
first_run=True

In [2]:
if first_run==True:
  !pip install numpy scipy
  !pip install resampy tensorflow six
  !pip install pysoundfile 
  #from mp3 to wav
  !pip install pydub



In [0]:
if first_run==True:
  import os
  import tensorflow as tf
  import numpy as np
  from pydub import AudioSegment
  from scipy.io.wavfile import read 

In [4]:
if first_run==True:

#clone repository
  !git clone https://github.com/tensorflow/models.git
   # Grab the VGGish model
  !curl -O https://storage.googleapis.com/audioset/vggish_model.ckpt
  !curl -O https://storage.googleapis.com/audioset/vggish_pca_params.npz

  # Copy the source files to the current directory.
  !cp models/research/audioset/* .

  #smoke test
  #from vggish_smoke_test import *

Cloning into 'models'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 24145 (delta 0), reused 1 (delta 0), pack-reused 24136
Receiving objects: 100% (24145/24145), 506.86 MiB | 32.34 MiB/s, done.
Resolving deltas: 100% (14291/14291), done.
Checking out files: 100% (2776/2776), done.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  277M  100  277M    0     0  33.3M      0  0:00:08  0:00:08 --:--:-- 50.2M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 73020  100 73020    0     0   369k      0 --:--:-- --:--:-- --:--:--  369k


In [5]:
if first_run==True:
  import vggish_slim
  import vggish_params
  import vggish_input
  import vggish_postprocess

def CreateVGGishNetwork(hop_size=0.96):   # Hop size is in seconds.
  """Define VGGish model, load the checkpoint, and return a dictionary that points
  to the different tensors defined by the model.
  """
  vggish_slim.define_vggish_slim()
  checkpoint_path = 'vggish_model.ckpt'
  vggish_params.EXAMPLE_HOP_SECONDS = hop_size
  
  vggish_slim.load_vggish_slim_checkpoint(sess, checkpoint_path)

  features_tensor = sess.graph.get_tensor_by_name(
      vggish_params.INPUT_TENSOR_NAME)
  embedding_tensor = sess.graph.get_tensor_by_name(
      vggish_params.OUTPUT_TENSOR_NAME)

  layers = {'conv1': 'vggish/conv1/Relu',
            'pool1': 'vggish/pool1/MaxPool',
            'conv2': 'vggish/conv2/Relu',
            'pool2': 'vggish/pool2/MaxPool',
            'conv3': 'vggish/conv3/conv3_2/Relu',
            'pool3': 'vggish/pool3/MaxPool',
            'conv4': 'vggish/conv4/conv4_2/Relu',
            'pool4': 'vggish/pool4/MaxPool',
            'fc1': 'vggish/fc1/fc1_2/Relu',
            'fc2': 'vggish/fc2/Relu',
            'embedding': 'vggish/embedding',
            'features': 'vggish/input_features',
         }
  g = tf.get_default_graph()
  for k in layers:
    layers[k] = g.get_tensor_by_name( layers[k] + ':0')
    
  return {'features': features_tensor,
          'embedding': embedding_tensor,
          'layers': layers,
         }


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [0]:
# def ProcessWithVGGish(vgg, x, sr):
#   '''Run the VGGish model, starting with a sound (x) at sample rate
#   (sr). Return a whitened version of the embeddings. Sound must be scaled to be
#   floats between -1 and +1.'''

#   # Produce a batch of log mel spectrogram examples.
#   input_batch = vggish_input.waveform_to_examples(x, sr)
#   # print('Log Mel Spectrogram example: ', input_batch[0])
#   print(input_batch.dtype,"input_batch")
#   input_batch=input_batch.astype(np.float32)
#   print(input_batch.dtype,"input_batch")

  
#   [embedding_batch] = sess.run([vgg['embedding']],
#                                feed_dict={vgg['features']: input_batch})

#   #Postprocess the results to produce whitened quantized embeddings.
#   pca_params_path = 'vggish_pca_params.npz'

#   pproc = vggish_postprocess.Postprocessor(pca_params_path)
#   postprocessed_batch = pproc.postprocess(embedding_batch)
#   print('Postprocessed VGGish embedding: ', postprocessed_batch[0])
#   return postprocessed_batch[0],embedding_batch



In [6]:

if first_run==True:
  from google.colab import auth
  auth.authenticate_user()

if not wav_files_exist:

  

Copying gs://deep_learning_enis/speech_audio_understanding/data.tar.gz...
Exception in UIThread: 
^C
tar (child): data.tar.gz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now


In [9]:
# do we have wav version of files
wav_files_exist=True

!rm -rf ./data/split_00\:10_wav/
!mkdir -p "./data/split_00:10_wav/"

if first_run==True and not wav_files_exist:
  # Download the file from a given Google Cloud Storage bucket.
  !gsutil cp gs://deep_learning_enis/speech_audio_understanding/data.tar.gz ./

  !tar xzf data.tar.gz
  

  mp3_data_files=os.listdir("./data/split_00:10/")
  error_file=mp3_data_files.index("._FSHCK1_20160726_190204_1440m_00s__1500m_00s_52m_00s__54m_00s_00m_20s__00m_30s.mp3")
  del mp3_data_files[error_file]
  
  for i,mp3_file in enumerate(mp3_data_files):
    sound = AudioSegment.from_mp3("./data/split_00:10/"+mp3_file)
    mp3_file_name=mp3_file.split(".")[0]
    sound.export("./data/split_00:10_wav/"+mp3_file_name+".wav", format="wav")
    if i%500==0:
      print(i/len(mp3_data_files))
  # upload wav files
  !tar -zcvf data_wav.tar.gz "./data/split_00:10_wav"
  !gsutil cp data_wav.tar.gz gs://deep_learning_enis/speech_audio_understanding


elif wav_files_exist:
    # Download the file from a given Google Cloud Storage bucket.
  !gsutil cp gs://deep_learning_enis/speech_audio_understanding/data_wav.tar.gz ./

  !tar xzf data_wav.tar.gz

wav_data_files=os.listdir("./data/split_00:10_wav/")


Copying gs://deep_learning_enis/speech_audio_understanding/data_wav.tar.gz...
| [1 files][ 10.3 GiB/ 10.3 GiB]   73.6 MiB/s                                   
Operation completed over 1 objects/10.3 GiB.                                     


3663

In [10]:
tf.reset_default_graph()
sess = tf.Session()

vgg = CreateVGGishNetwork()

pca_params_path = 'vggish_pca_params.npz'
pproc = vggish_postprocess.Postprocessor(pca_params_path)


batch_size=128
sr=44100

# OPTION 1 STORE ALL OUTPUTS:
# all_data = np.array([], dtype=np.float64).reshape(0,96,64)

# data_file_indexes=[]
# counter=0

# wav_data_files=os.listdir("./data/split_00:10_wav/")

# for i,wav_file in enumerate(wav_data_files):
#   rate,sound = read("./data/split_00:10_wav/"+wav_file)
#   sound=np.array(sound,dtype=float)

#   sound = vggish_input.waveform_to_examples(sound, sr)
#   data_file_indexes.append((counter,counter+len(sound)))
#   counter+=len(sound)
#   all_data=np.concatenate((all_data,sound))

#   if i%500==0:
#     print(i/len(wav_data_files))

# # all_data=np.load('all_data.npy')

# embeddings = np.array([], dtype=np.float32).reshape(0,128)
# postprocessed = np.array([], dtype=np.float32).reshape(0,128)


# for batch_index in range(0,all_data.shape[0],batch_size):
#   if (batch_index+batch_size) <all_data.shape[0]:
#     input_batch = all_data[batch_index:batch_index+batch_size]
    
#     [embedding_batch] = sess.run([vgg['embedding']],
#                              feed_dict={vgg['features']: input_batch})
#     embeddings=np.concatenate((embeddings,embedding_batch))
    
#     postprocessed_batch = pproc.postprocess(embedding_batch)
#     postprocessed=np.concatenate((postprocessed,postprocessed_batch))
#     if batch_index%(batch_size*20)==0:
#       print(batch_index/all_data.shape[0])

# np.save("all_data2.npy",all_data)
# np.save("data_file_indexes2.npy",data_file_indexes)
# np.save("embeddings2.npy",embeddings)
# np.save("postprocessed2.npy",postprocessed)

# !gsutil cp all_data2.npy gs://deep_learning_enis/speech_audio_understanding
# !gsutil cp embeddings2.npy gs://deep_learning_enis/speech_audio_understanding
# !gsutil cp postprocessed2.npy gs://deep_learning_enis/speech_audio_understanding
# !gsutil cp data_file_indexes2.npy gs://deep_learning_enis/speech_audio_understanding



Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from vggish_model.ckpt


In [12]:
#wav file: indexes of signal that it generated (10,22) means embeddings[10,22] belong to that wav file
data_file_indexes={}
counter=0

embeddings = np.array([], dtype=np.float32).reshape(0,128)
postprocessed = np.array([], dtype=np.float32).reshape(0,128)



for batch_index in range(0,len(wav_data_files),batch_size):
  if (batch_index+batch_size) <len(wav_data_files):
    a_batch=wav_data_files[batch_index:batch_index+batch_size]
  else:
    a_batch=wav_data_files[batch_index:]
  
  # wav to one second signals
  batch_processed = np.array([], dtype=np.float64).reshape(0,96,64)
  for i,wav_file in enumerate(a_batch):
      rate,sound = read("./data/split_00:10_wav/"+wav_file)
      sound=np.array(sound,dtype=float)

      sound = vggish_input.waveform_to_examples(sound, sr)
      data_file_indexes[wav_file]=(counter,counter+len(sound))
      counter+=len(sound)
      batch_processed=np.concatenate((batch_processed,sound))
  # signals to tensors
  [embedding_batch] = sess.run([vgg['embedding']],
                           feed_dict={vgg['features']: batch_processed})
  
  embeddings=np.concatenate((embeddings,embedding_batch))
  postprocessed_batch = pproc.postprocess(embedding_batch)
  postprocessed=np.concatenate((postprocessed,postprocessed_batch))
  if batch_index%(batch_size*20)==0:
    print(batch_index/len(wav_data_files))




0.0
0.26666666666666666
0.5333333333333333
0.8


In [13]:
np.save("data_file_indexes2.npy",data_file_indexes)
np.save("embeddings2.npy",embeddings)
np.save("postprocessed2.npy",postprocessed)

!gsutil cp embeddings2.npy gs://deep_learning_enis/speech_audio_understanding
!gsutil cp postprocessed2.npy gs://deep_learning_enis/speech_audio_understanding
!gsutil cp data_file_indexes2.npy gs://deep_learning_enis/speech_audio_understanding


Copying file://embeddings2.npy [Content-Type=application/octet-stream]...
-
Operation completed over 1 objects/46.9 MiB.                                     
Copying file://postprocessed2.npy [Content-Type=application/octet-stream]...
-
Operation completed over 1 objects/46.9 MiB.                                     
Copying file://data_file_indexes2.npy [Content-Type=application/octet-stream]...
/ [1 files][973.2 KiB/973.2 KiB]                                                
Operation completed over 1 objects/973.2 KiB.                                    


In [0]:
first_run=False



In [0]:
# # dates=[]
# all_files=[]
# for name in data_files:
#   file_id=name
#   name=name.split("_")
# #   if len(name)!=18:
# #     print(name)
#   site_name=name[0]
#   date=name[1]
#   year=date[0:4]
#   month=date[4:6]
#   day=date[6:8]
  
#   site_id=name[2]

# #   if year=="2013" and site_name=="UMIAT":
# #     print(name)
#   all_files.append(name)
# #   if len(date)!=len("20160809"):
# #     print(name)
# #   dates.append((year,month,day))
# # site_name,date(YYYY,MM,DD),hour(HHMMSS),total_running_time_start(M,S),total_running_time_end(M,S), start-end(M,S,) start-end (M,S),start-end(M,S,) start-end (M,S)